In [14]:
import gradio as gr
import torch
import numpy as np
import pandas as pd
import joblib
import sys
import os

# Add the root directory to sys.path otherwise import fails
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from model.network import ReNN

# --- Constants
MODEL_PATH = "../model/model.pth"
SCALER_X_PATH = "../model/scaler_X.pkl"
SCALER_Y_PATH = "../model/scaler_y.pkl"

# --- Load scalers
if os.path.exists(SCALER_X_PATH) and os.path.exists(SCALER_Y_PATH):
    scaler_X = joblib.load(SCALER_X_PATH)
    scaler_y = joblib.load(SCALER_Y_PATH)
else:
    raise FileNotFoundError("Scaler files not found. Please train the model first.")

# --- Dummy input to get input size
input_size = scaler_X.mean_.shape[0]

# --- Load model
model = ReNN(input_size)
if os.path.exists(MODEL_PATH):
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()
    print("Loaded trained model.")
else:
    print("Model not found. Using untrained model.")

def predict_yield(rainfall, temperature, elevation, fertilizer, shade, soil_type):
    # One-hot encode soil type
    soil_loamy = 1 if soil_type == "Loamy" else 0
    soil_clay = 1 if soil_type == "Clay" else 0
    
    input_data = np.array([[rainfall, temperature, elevation, fertilizer, shade, soil_loamy, soil_clay]])
    
    # Scale input
    X_scaled = scaler_X.transform(input_data)
    X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
    
    with torch.no_grad():
        y_pred = model(X_tensor)
    
    # Inverse transform prediction
    y_unscaled = scaler_y.inverse_transform(y_pred.numpy())
    
    return round(float(y_unscaled[0][0]), 2)

inputs = [
    gr.Slider(500, 3000, label="Rainfall (mm/year)"),
    gr.Slider(10, 35, label="Avg Temperature (°C)"),
    gr.Slider(500, 2000, label="Elevation (m)"),
    gr.Slider(0, 100, label="Fertilizer (kg/ha)"),
    gr.Slider(0, 100, label="Shade Trees (%)"),
    gr.Dropdown(["Loamy", "Clay"], label="Soil Type"),
]

output = gr.Number(label="Predicted Yield (kg/ha)")

demo = gr.Interface(fn=predict_yield, inputs=inputs, outputs=output, title="YieldBrew: Coffee Yield Estimator")
demo.launch()



C:\Users\nicho\AppData\Local\Temp\ipykernel_19920\2532888983.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


Loaded trained model.
* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
